In [1]:
import pandas as pd

In [12]:
def get_stand_benchmark_2035(building, stand_benchmarks, building_data, baseline):
    benchmarks_2035 = stand_benchmarks.loc['2031-2035']
    building_data_row = building_data[building_data['OSEBuildingID'] == building['OSEBuildingID']].iloc[0]
    
    if building['LargestPropertyUseType OSE'] == 'nan' or pd.isna(building['LargestPropertyUseType OSE']):
        largest_ghgit = building_data_row['percent_sqft_1st'] * baseline
    else:
        largest_ghgit = building_data_row['percent_sqft_1st'] * benchmarks_2035[building['LargestPropertyUseType OSE']]
        
    if building['SecondLargestPropertyUseType OSE'] == 'nan' or pd.isna(building['SecondLargestPropertyUseType OSE']):
        second_ghgit = building_data_row['percent_sqft_2nd'] * baseline
    else:
        second_ghgit = building_data_row['percent_sqft_2nd'] * benchmarks_2035[building['SecondLargestPropertyUseType OSE']]
    
    if building['ThirdLargestPropertyUseType OSE'] == 'nan' or pd.isna(building['ThirdLargestPropertyUseType OSE']):
        third_ghgit = building_data_row['percent_sqft_3rd'] * baseline
    else:
        third_ghgit = building_data_row['percent_sqft_3rd'] * benchmarks_2035[building['ThirdLargestPropertyUseType OSE']]
    
    return largest_ghgit + second_ghgit + third_ghgit

In [2]:
def eligible_for_exception_3_orig(building, scen, stand_benchmarks, building_data):
    '''
    This was used in calculations from mid-July
    '''
    benchmarks_2035 = stand_benchmarks.loc['2031-2035']

    building_data_row = building_data[building_data['OSEBuildingID'] == building['OSEBuildingID']].iloc[0]
    baseline_2035 = scen[(scen['year'] == 2035) & (scen['OSEBuildingID'] == building['OSEBuildingID'])].iloc[0]['expected_baseline_ghgi']
    
    ghgit_2035 = get_stand_benchmark_2035(building, stand_benchmarks, building_data, baseline_2035)
    baseline_ghgi = scen[(scen['year'] == 2027) & (scen['OSEBuildingID'] == building['OSEBuildingID'])].iloc[0]['expected_baseline_ghgi']
    
    #print(baseline_ghgi, ghgit_2035 * 3.5)
    return baseline_ghgi > ghgit_2035 * 3.5

In [3]:
def eligible_for_exception_3(building, scen, stand_benchmarks, building_data):
    '''
    According to 150% standard mentioned in report as of 8/1
    '''
    benchmarks_2035 = stand_benchmarks.loc['2031-2035']

    building_data_row = building_data[building_data['OSEBuildingID'] == building['OSEBuildingID']].iloc[0]
    baseline_2035 = scen[(scen['year'] == 2035) & (scen['OSEBuildingID'] == building['OSEBuildingID'])].iloc[0]['expected_baseline_ghgi']
    
    ghgit_2035 = get_stand_benchmark_2035(building, stand_benchmarks, building_data, baseline_2035)
    baseline_ghgi = scen[(scen['year'] == 2027) & (scen['OSEBuildingID'] == building['OSEBuildingID'])].iloc[0]['expected_baseline_ghgi']
    
    return baseline_ghgi >= ghgit_2035 * 1.5

In [5]:
building_data = pd.read_csv('../data/input_data/building_data_clean.csv')

In [16]:
scen = pd.read_csv('../data/results_data/baseline_calcs/scen_2_calcs.csv')

In [17]:
scen.head()

,Unnamed: 0,OSEBuildingID,BuildingName,Total_sqft,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,0,1,MAYFLOWER PARK HOTEL,88434,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,0.0
1,1,2,PARAMOUNT HOTEL,88502,C,Hotel,NaN,Restaurant,2027.0,2.843616e+05,3.213052,3.213052,3.213052,2.843616e+05,No,0.0
2,2,3,WESTIN HOTEL (Parent Building),756493,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.900095,2.900095,2.900095,2.193901e+06,No,0.0
3,3,5,HOTEL MAX,61320,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,0.0
4,4,8,WARWICK SEATTLE HOTEL,123445,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,4.720742,4.720742,4.720742,5.827520e+05,No,0.0


In [7]:
stand_benchmarks = pd.read_csv('../data/input_data/raw_data/scen 2 stand targets.csv')

In [11]:
stand_benchmarks.head()

,Scenario 2 Provisional Targets (Year),College/University,Entertainment/Public Assembly,Fire/Police Station,Hospital,Hotel,K-12 School,Laboratory,Multifamily Housing2,Non-Refrigerated Warehouse,...,#REF!.15,#REF!.16,#REF!.17,#REF!.18,#REF!.19,#REF!.20,#REF!.21,#REF!.22,Unnamed: 238,Unnamed: 239
0,2031-2035,2.69,1.18,2.23,4.68,2.06,0.95,6.30,0.89,0.77,...,3.22,0.98,1.16,5.73,1.03,0.31,2.11,1.36,3.42,1.2
1,2036-2040,1.57,0.69,1.30,2.73,1.20,0.56,3.68,0.63,0.45,...,1.88,0.57,0.68,3.34,0.60,0.18,1.23,0.79,2.00,0.7
2,2041-2045,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.37,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3,2046-2050,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [13]:
stand_benchmarks.set_index('Scenario 2 Provisional Targets (Year)', inplace=True)

In [19]:
stand_benchmarks['Multifamily Housing'] = stand_benchmarks['Multifamily Housing2']

In [20]:
building_data['orig_exception_3'] = building_data.apply(lambda building: eligible_for_exception_3_orig(building, scen, stand_benchmarks, building_data), axis=1)

In [24]:
building_data['orig_exception_3'].sum()

194

In [25]:
building_data['exception_3'] = building_data.apply(lambda building: eligible_for_exception_3(building, scen, stand_benchmarks, building_data), axis=1)

In [26]:
building_data['exception_3'].sum()

845

In [27]:
building_data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'OSEBuildingID', 'BuildingName',
       'BuildingType', 'Type_of_Bulding', 'PropertyGFATotal',
       'PropertyGFABuilding(s)', 'PropertyGFAParking', 'Total_sqft',
       'percent_sqft_1st', 'percent_sqft_2nd', 'percent_sqft_3rd',
       'LargestPropertyUseType', 'LargestPropertyUseType OSE',
       'LargestPropertyUseTypeGFA', 'LargestPropertyUseTypeGFA Analysis',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseType OSE',
       'SecondLargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseTypeGFA Analysis',
       'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseType OSE',
       'ThirdLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseTypeGFA Analysis', 'Electricity(kBtu)',
       'Steam(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity', 'Total_GFA', 'sq_ft_classification',
       'orig_exception_3', 'exception_3'],
      dtype='object')

In [28]:
building_data.columns = ['Unnamed: 0.1', 'Unnamed: 0', 'OSEBuildingID', 'BuildingName',
       'BuildingType', 'Type_of_Bulding', 'PropertyGFATotal',
       'PropertyGFABuilding(s)', 'PropertyGFAParking', 'Total_sqft',
       'percent_sqft_1st', 'percent_sqft_2nd', 'percent_sqft_3rd',
       'LargestPropertyUseType', 'LargestPropertyUseType OSE',
       'LargestPropertyUseTypeGFA', 'LargestPropertyUseTypeGFA Analysis',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseType OSE',
       'SecondLargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseTypeGFA Analysis',
       'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseType OSE',
       'ThirdLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseTypeGFA Analysis', 'Electricity(kBtu)',
       'Steam(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions',
       'GHGEmissionsIntensity', 'Total_GFA', 'sq_ft_classification',
       'exception_3_w_3.5x_target', 'exception_3_w_1.5x_target']

In [29]:
building_data.to_csv('../data/results_data/buildings_subject_to_exception_3.csv')